# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.30.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-147669
aml-quickstarts-147669
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


In [4]:
# choose a name for experiment
experiment_name = 'heart-experiment'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-experiment,quick-starts-ws-147669,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster-exp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
key = "Heart-failure"
description_text = "Heart Failure Prediction DataSet from Kaggle"

# Create AML Dataset and register it into Workspace
example_data = 'https://raw.githubusercontent.com/santosh-gatech/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv'
dataset = Dataset.Tabular.from_delimited_files(example_data)        
#Register Dataset in Workspace
dataset = dataset.register(workspace=ws,
                            name=key,
                            description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [13]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_ouput=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-experiment,AutoML_81b2b526-2f3b-42e6-bedf-bc0687eeaed6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [17]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
heart-experiment,AutoML_81b2b526-2f3b-42e6-bedf-bc0687eeaed6,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missin

{'runId': 'AutoML_81b2b526-2f3b-42e6-bedf-bc0687eeaed6',
 'target': 'cpu-cluster-exp',
 'status': 'Completed',
 'startTimeUtc': '2021-06-22T06:25:54.185068Z',
 'endTimeUtc': '2021-06-22T06:46:04.211911Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster-exp',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-experiment","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-147669","workspace_name":"quick-starts-ws-147669","region":"southcentralus","compute_target":"cpu-cluster-exp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_model, fitted_model = remote_run.get_output()
best_model.get_metrics()

{'matthews_correlation': 0.7275926385633272,
 'AUC_weighted': 0.9189431343785464,
 'f1_score_micro': 0.8763218390804598,
 'recall_score_weighted': 0.8763218390804598,
 'f1_score_weighted': 0.8720932016548396,
 'recall_score_macro': 0.8487553266400519,
 'precision_score_weighted': 0.889871678591595,
 'average_precision_score_micro': 0.9255012812300241,
 'weighted_accuracy': 0.894344335592302,
 'average_precision_score_macro': 0.909851704919947,
 'recall_score_micro': 0.8763218390804598,
 'f1_score_macro': 0.8523696072483118,
 'balanced_accuracy': 0.8487553266400519,
 'log_loss': 0.35678780556620343,
 'AUC_micro': 0.922750693618708,
 'precision_score_macro': 0.8814139299551199,
 'accuracy': 0.8763218390804598,
 'norm_macro_recall': 0.6975106532801041,
 'average_precision_score_weighted': 0.9308416648773268,
 'AUC_macro': 0.9189431343785464,
 'precision_score_micro': 0.8763218390804598,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_81b2b526-2f3b-42e6-bedf-bc0687eeaed6_38/

In [22]:
print("best automl model:\n{}".format(best_model))


best automl model:
Run(Experiment: heart-experiment,
Id: AutoML_81b2b526-2f3b-42e6-bedf-bc0687eeaed6_38,
Type: azureml.scriptrun,
Status: Completed)


In [23]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.7368421052631579, reg_lambda=0.7368421052631579, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]))],
         verbose=False)


In [20]:
#TODO: Save the best model
best_model_metrics = best_model.get_metrics()
best_model.register_model(
    model_name='best_automl_model', 
    model_path='/outputs', 
    properties={'Accuracy': best_model_metrics['accuracy']}, 
    tags={'Training context': 'Auto ML'})

Model(workspace=Workspace.create(name='quick-starts-ws-147669', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-147669'), name=best_automl_model, id=best_automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8763218390804598'})

In [21]:
model_name = best_model.properties['model_name']
model_name

'AutoML81b2b526238'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import sklearn
import json

# Register model

ml_model = remote_run.register_model(
    model_name=model_name,
    description='Best AutoML model' 
)

print(ml_model.name)

AutoML81b2b526238


In [25]:
env = best_model.get_environment()
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [26]:
# Inference config

infer_config = InferenceConfig(entry_script='score.py', environment=env)
service_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    enable_app_insights=True
)

service_name = 'heart-project-service'


In [27]:
from azureml.core import Webservice, Model

deploy_service = Model.deploy(
    ws,
    service_name,
    [ml_model],
    infer_config,
    service_config
)

In [29]:
deploy_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-22 09:13:21+00:00 Creating Container Registry if not exists.
2021-06-22 09:13:32+00:00 Registering the environment.
2021-06-22 09:13:33+00:00 Use the existing image.
2021-06-22 09:13:33+00:00 Generating deployment configuration.
2021-06-22 09:13:34+00:00 Submitting deployment to compute.
2021-06-22 09:13:37+00:00 Checking the status of deployment heart-project-service..
2021-06-22 09:17:31+00:00 Checking the status of inference endpoint heart-project-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [30]:
print(deploy_service.state)

Healthy


In [33]:
service_url = deploy_service.scoring_uri
print(service_url)

http://7fd14725-046b-4c61-9a70-87ef2bad6776.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
%run endpoint.py

{"result": [1, 1]}


In [37]:
test_data = dataset.to_pandas_dataframe().dropna()
test_data_sample = test_data.sample(5)

In [38]:
y_truth = test_data_sample.pop("DEATH_EVENT")
sample_json_data = json.dumps(
    {'data': test_data_sample.to_dict(orient='records')}
)
print(sample_json_data)

{"data": [{"age": 42.0, "anaemia": 1, "creatinine_phosphokinase": 250, "diabetes": 1, "ejection_fraction": 15, "high_blood_pressure": 0, "platelets": 213000.0, "serum_creatinine": 1.3, "serum_sodium": 136, "sex": 0, "smoking": 0, "time": 65}, {"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 292, "diabetes": 1, "ejection_fraction": 35, "high_blood_pressure": 0, "platelets": 850000.0, "serum_creatinine": 1.3, "serum_sodium": 142, "sex": 1, "smoking": 1, "time": 88}, {"age": 61.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 147000.0, "serum_creatinine": 1.2, "serum_sodium": 141, "sex": 1, "smoking": 0, "time": 237}, {"age": 50.0, "anaemia": 1, "creatinine_phosphokinase": 115, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 0, "platelets": 189000.0, "serum_creatinine": 0.8, "serum_sodium": 139, "sex": 1, "smoking": 0, "time": 146}, {"age": 43.0, "anaemia": 1, "creatinine_phosphokinase": 

TODO: In the cell below, print the logs of the web service and delete the service

In [39]:
service_output = deploy_service.run(sample_json_data)
print("Predicted values: \n{}".format(service_output))
print("True Values: \n{}".format(y_truth.values))

Predicted values: 
{"result": [1, 0, 0, 0, 0]}
True Values: 
[1 0 0 0 0]


In [40]:
deploy_service.get_logs()

'2021-06-22T09:17:16,445053900+00:00 - rsyslog/run \n2021-06-22T09:17:16,445979100+00:00 - iot-server/run \n2021-06-22T09:17:16,454940900+00:00 - gunicorn/run \n2021-06-22T09:17:16,546908900+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [41]:
deploy_service.delete()


In [42]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

